In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import helmeos
from matplotloom import Loom
from glob import glob
from joblib import Parallel, delayed
import joblib
from tqdm import tqdm

In [3]:
def get_kappa_value(filepath):
    part_after_kappa = filepath.split('_kappa_')[-1]
    kappa_str = part_after_kappa.split('_alpha_')[0]
    return float(kappa_str)

In [4]:
gconst = 6.67430e-8
clight = 2.99792458e10
solar = 1.98847e33
vcgs2code = 1/clight
lencgs2code = (clight**2)/(solar*gconst)
masscgs2code = (1.0e0/solar)
rhocgs2code = (masscgs2code/lencgs2code**3)
tcgs2code = (clight**3)/(solar*gconst)
mu_0 = 1.25663706212e-6
amp2code = (mu_0*1.0e5*masscgs2code*lencgs2code)**(0.5e0)/tcgs2code
gauss2code = 1.0e-1*masscgs2code/amp2code/tcgs2code**2
energycgs2code = (1.0E0/clight**2)
me2 = 9.1093837015e-28*masscgs2code
mb2 = 1.66053906660e-24*masscgs2code
ye = 0.5e0
h_bar = (1.054571817e-27)*(lencgs2code**2*masscgs2code/tcgs2code)
amax = (me2**4)/(2.4e1*np.pi**2*h_bar**3)
bmax = (mb2*me2**3)/(3.0e0*np.pi**2*h_bar**3*ye)

In [5]:
1/tcgs2code

4.925639893961039e-06

In [6]:
data_paths = glob("/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_*_AxisRatio_0.699*/")
data_paths = sorted(data_paths, key=lambda filepath: float(filepath.split('_kappa_')[-1].split('_alpha_')[0]))
data_paths

['/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.000E+00_alpha_0.000E+00/',
 '/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.100E-04_alpha_0.000E+00/',
 '/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.300E-04_alpha_0.000E+00/',
 '/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.500E-04_alpha_0.000E+00/',
 '/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.700E-04_alpha_0.000E+00/',
 '/home/cnchong/Codes/cumc3d/model/Type_Ia/runs/Output_CentralDensity_9.477_AxisRatio_0.699_kappa_0.100E-03_alpha_0.000E+00/']

In [7]:
nf_list = []
for chosen in data_paths:
    working_directory = chosen+'outfile/'
    profile_directory = chosen+'profile'
    files = []
    for f in glob(working_directory+'/*.hdf5'):
        files.append(f)
    n = len(files)-1
    nf_list.append(n)
n = np.min(nf_list)

time_list = []
for i in range(n):
    filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
    f = h5py.File(filename, "r")
    time = f['time'][0]
    time_list.append(time)
    f.close()

In [8]:
def only_plot_at_i(i):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=len(data_paths), layout='constrained', figsize=(24,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    
    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        profile_directory = chosen+'profile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        primitive = f['primitive'][:].T

        # Face value
        x1f = np.loadtxt(profile_directory+'/hydro_x1_fgrid.dat').T
        x3f = np.loadtxt(profile_directory+'/hydro_x3_fgrid.dat').T
        # Center value
        x1f = x1f [3:-3]*lencgs2code # Remove BC
        x1c = (x1f[1:]+x1f[:-1])/2 #mid pt
        x3f = x3f[3:-3]*lencgs2code # Remove BC
        x3c = (x3f[1:]+x3f[:-1])/2 #mid pt

        xx,zz= np.meshgrid(x1c,x3c,indexing='ij')

        xxf, zzf = np.meshgrid(x1f,x3f,indexing='ij')

        kappa = get_kappa_value(chosen)

        dens = ax[0,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(primitive[0,1:-1,1,1:-1]/rhocgs2code),cmap='inferno')
        plt.colorbar(dens,ax=ax[0,j])
        if j == 0:
            ax[0,j].set_ylabel(r'$\log \rho$')
        ax[0,j].set_title(r'$\kappa =$'+str('{:.2e}'.format(kappa)))

        pdens = ax[1,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(0.5*primitive[0,1:-1,1,1:-1]*(primitive[1,1:-1,1,1:-1]**2+primitive[3,1:-1,1,1:-1]**2)/(rhocgs2code*vcgs2code**2)),cmap='inferno')
        plt.colorbar(pdens,ax=ax[1,j])
        if j == 0:
            ax[1,j].set_ylabel(r'$\log \rho v_r^2/2$')

        rdens = ax[2,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(0.5*primitive[0,1:-1,1,1:-1]*primitive[2,1:-1,1,1:-1]**2/(rhocgs2code*vcgs2code**2)),cmap='inferno')
        plt.colorbar(rdens,ax=ax[2,j])
        if j == 0:
            ax[2,j].set_ylabel(r'$\log \rho v_\phi^2/2$')

def plot_at_i(i, loom):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=len(data_paths), layout='constrained', figsize=(24,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    
    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        profile_directory = chosen+'profile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        primitive = f['primitive'][:].T

        # Face value
        x1f = np.loadtxt(profile_directory+'/hydro_x1_fgrid.dat').T
        x3f = np.loadtxt(profile_directory+'/hydro_x3_fgrid.dat').T
        # Center value
        x1f = x1f [3:-3]*lencgs2code # Remove BC
        x1c = (x1f[1:]+x1f[:-1])/2 #mid pt
        x3f = x3f[3:-3]*lencgs2code # Remove BC
        x3c = (x3f[1:]+x3f[:-1])/2 #mid pt

        xx,zz= np.meshgrid(x1c,x3c,indexing='ij')

        xxf, zzf = np.meshgrid(x1f,x3f,indexing='ij')

        kappa = get_kappa_value(chosen)

        dens = ax[0,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(primitive[0,1:-1,1,1:-1]/rhocgs2code),cmap='inferno',vmin=3,vmax=10)
        plt.colorbar(dens,ax=ax[0,j])
        if j == 0:
            ax[0,j].set_ylabel(r'$\log \rho$')
        ax[0,j].set_title(r'$\kappa =$'+str('{:.2e}'.format(kappa)))

        pdens = ax[1,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(0.5*primitive[0,1:-1,1,1:-1]*(primitive[1,1:-1,1,1:-1]**2+primitive[3,1:-1,1,1:-1]**2)/(rhocgs2code*vcgs2code**2)),cmap='inferno',vmin=15,vmax=27)
        plt.colorbar(pdens,ax=ax[1,j])
        if j == 0:
            ax[1,j].set_ylabel(r'$\log \rho v_r^2/2$')

        rdens = ax[2,j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(0.5*primitive[0,1:-1,1,1:-1]*primitive[2,1:-1,1,1:-1]**2/(rhocgs2code*vcgs2code**2)),cmap='inferno',vmin=15,vmax=27)
        plt.colorbar(rdens,ax=ax[2,j])
        if j == 0:
            ax[2,j].set_ylabel(r'$\log \rho v_\phi^2/2$')
    loom.save_frame(fig, i)

def only_Bplot_at_i(i):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=len(data_paths), layout='constrained', figsize=(24,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    
    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        bfields = f['bfield'][:].T

        kappa = get_kappa_value(chosen)

        B2 = bfields[0,1:-1,1,1:-1]**2+bfields[1,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        pB2 = bfields[0,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        rB2 = bfields[1,1:-1,1,1:-1]**2

        B = ax[0,j].imshow(np.log10(np.sqrt(B2)/gauss2code).T, aspect='auto')
        plt.colorbar(B,ax=ax[0,j])
        if j == 0:
            ax[0,j].set_ylabel(r'$\log B$')
        ax[0,j].set_title(r'$\kappa =$'+str('{:.2e}'.format(kappa)))

        pB = ax[1,j].imshow(np.log10(np.sqrt(pB2)/gauss2code).T, aspect='auto')
        plt.colorbar(pB,ax=ax[1,j])
        if j == 0:
            ax[1,j].set_ylabel(r'$\log B_p$')

        rB = ax[2,j].imshow(np.log10(np.sqrt(rB2)/gauss2code).T, aspect='auto')
        plt.colorbar(rB,ax=ax[2,j])
        if j == 0:
            ax[2,j].set_ylabel(r'$\log B_t$')

def Bplot_at_i(i, loom):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=len(data_paths), layout='constrained', figsize=(24,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    
    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        bfields = f['bfield'][:].T

        kappa = get_kappa_value(chosen)

        B2 = bfields[0,1:-1,1,1:-1]**2+bfields[1,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        pB2 = bfields[0,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        rB2 = bfields[1,1:-1,1,1:-1]**2
        
        B = ax[0,j].imshow(np.log10(np.sqrt(B2)/gauss2code).T, aspect='auto',vmin=-10, vmax=13)
        plt.colorbar(B,ax=ax[0,j])
        if j == 0:
            ax[0,j].set_ylabel(r'$\log B$')
        ax[0,j].set_title(r'$\kappa =$'+str('{:.2e}'.format(kappa)))

        pB = ax[1,j].imshow(np.log10(np.sqrt(pB2)/gauss2code).T, aspect='auto',vmin=-10, vmax=13)
        plt.colorbar(pB,ax=ax[1,j])
        if j == 0:
            ax[1,j].set_ylabel(r'$\log B_p$')

        rB = ax[2,j].imshow(np.log10(np.sqrt(rB2)/gauss2code).T, aspect='auto',vmin=-10, vmax=13)
        plt.colorbar(rB,ax=ax[2,j])
        if j == 0:
            ax[2,j].set_ylabel(r'$\log B_t$')

    loom.save_frame(fig, i)

def Blineplot_at_i(i, loom):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=1, layout='constrained', figsize=(12,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    kappa_list = []

    for chosen in data_paths:
        kappa_list.append(get_kappa_value(chosen))

    for j, chosen in enumerate(data_paths[1:]):
        working_directory = chosen+'outfile/'
        profile_directory = chosen+'profile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        bfields = f['bfield'][:].T

        kappa = get_kappa_value(chosen)
        colors = plt.cm.viridis(np.array(kappa_list)/np.max(kappa_list))

        # Face value
        x1f = np.loadtxt(profile_directory+'/hydro_x1_fgrid.dat').T
        # Center value
        x1f = x1f [3:-3]*lencgs2code # Remove BC
        # x1c = (x1f[1:]+x1f[:-1])/2 #mid pt

        B2 = bfields[0,1:-1,1,1:-1]**2+bfields[1,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        pB2 = bfields[0,1:-1,1,1:-1]**2+bfields[2,1:-1,1,1:-1]**2
        rB2 = bfields[1,1:-1,1,1:-1]**2

        mid = int((B2.shape[1]-1)/2 )

        B = ax[0].plot(x1f, np.log10(np.sqrt(B2)[:,mid]/gauss2code), label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[0].set_ylabel(r'$\log B$')

        pB = ax[1].plot(x1f, np.log10(np.sqrt(pB2)[:,mid]/gauss2code), label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[1].set_ylabel(r'$\log B_p$')

        rB = ax[2].plot(x1f, np.log10(np.sqrt(rB2)[:,mid]/gauss2code), label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[2].set_ylabel(r'$\log B_t$')
        
        ax[0].legend(loc='upper right')
    
        loom.save_frame(fig, i)

def plineplot_at_i(i,loom):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=3, ncols=1, layout='constrained', figsize=(12,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    kappa_list = []

    for chosen in data_paths:
        kappa_list.append(get_kappa_value(chosen))

    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        profile_directory = chosen+'profile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        primitive = f['primitive'][:].T
        bfields = f['bfield'][:].T

        kappa = get_kappa_value(chosen)
        colors = plt.cm.viridis(np.array(kappa_list)/np.max(kappa_list))

        # Face value
        x1f = np.loadtxt(profile_directory+'/hydro_x1_fgrid.dat').T
        # Center value
        x1f = x1f [3:-3]*lencgs2code # Remove BC
        x1c = (x1f[1:]+x1f[:-1])/2 #mid pt

        mid = int((primitive[0,1:-1,1,1:-1].shape[1]-1)/2)

        dens = primitive[0,1:-1,1,1:-1][:,mid]/rhocgs2code
        pressure = primitive[4,1:-1,1,1:-1][:,mid]/energycgs2code

        Br = bfields[0,1:-1,1,1:-1]
        Bt = bfields[1,1:-1,1,1:-1]
        Bz = bfields[2,1:-1,1,1:-1]

        Br_line = (Br[:,mid] + Br[:,mid+1])/2
        Bt_line = (Bt[:,mid] + Bt[:,mid+1])/2
        Bz_line = (Bz[:,mid] + Bz[:,mid+1])/2

        Br_line_cell = (Br_line[1:] + Br_line[:-1])/2
        Bt_line_cell = (Bt_line[1:] + Bt_line[:-1])/2
        Bz_line_cell = (Bz_line[1:] + Bz_line[:-1])/2

        B2 = (Br_line_cell**2+Bt_line_cell**2+Bz_line_cell**2)/gauss2code**2

        beta = B2/(2*pressure)
        Nif = primitive[12,1:-1,1,1:-1][:,mid]

        ax[0].plot(x1c/lencgs2code*1e-5, np.log10(dens), label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[0].set_ylabel(r'$\log \rho$')

        ax[1].plot(x1c/lencgs2code*1e-5, np.log10(beta), label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[1].set_ylabel(r'$\log B^2/2P$')

        ax[2].plot(x1c/lencgs2code*1e-5, Nif, label=r'$\kappa =$'+str('{:.2e}'.format(kappa)),color=colors[j])
        if j == 0:
            ax[2].set_ylabel(r'$X_\text{Ni}$')
        
        ax[0].legend(loc='upper right')

        loom.save_frame(fig, i)

def Ni_mass_at_i(i,loom):
    np.seterr(divide = 'ignore') 
    time = time_list[i]
    fig, ax = plt.subplots(nrows=1, ncols=len(data_paths), layout='constrained', figsize=(24,6), sharex={"col"})
    fig.suptitle('At $t=$'+ str('{:.3f}'.format(time/tcgs2code))+' s, units in cgs')
    
    for j, chosen in enumerate(data_paths):
        working_directory = chosen+'outfile/'
        profile_directory = chosen+'profile/'
        filename = working_directory+"/rkiter-"+str(i)+"-nm.hdf5"
        f = h5py.File(filename, "r")
        primitive = f['primitive'][:].T

        # Face value
        x1f = np.loadtxt(profile_directory+'/hydro_x1_fgrid.dat').T
        x3f = np.loadtxt(profile_directory+'/hydro_x3_fgrid.dat').T
        # Center value
        x1f = x1f [3:-3]*lencgs2code # Remove BC
        x1c = (x1f[1:]+x1f[:-1])/2 #mid pt
        x3f = x3f[3:-3]*lencgs2code # Remove BC
        x3c = (x3f[1:]+x3f[:-1])/2 #mid pt

        xx,zz= np.meshgrid(x1c,x3c,indexing='ij')

        xxf, zzf = np.meshgrid(x1f,x3f,indexing='ij')

        kappa = get_kappa_value(chosen)

        dens = ax[j].pcolormesh(xxf/lencgs2code*1e-5,zzf/lencgs2code*1e-5,np.log10(primitive[0,1:-1,1,1:-1]*primitive[12,1:-1,1,1:-1]/rhocgs2code),cmap='inferno',vmin=3,vmax=10)
        plt.colorbar(dens,ax=ax[j])
        if j == 0:
            ax[j].set_ylabel(r'$\log \rho$')
        ax[j].set_title(r'$\kappa =$'+str('{:.2e}'.format(kappa)))

    loom.save_frame(fig, i)


In [ ]:
with Loom("parallel_pline.mp4", fps=10, parallel=True) as loom:
    Parallel(n_jobs=20)(
        delayed(plineplot_at_i)(i, loom)
        for i in tqdm(range(n))
    )

 44%|████▍     | 40/90 [00:04<00:06,  8.16it/s]

In [ ]:
with Loom("parallel_Ni.mp4", fps=10, parallel=True) as loom:
    Parallel(n_jobs=20)(
        delayed(Ni_mass_at_i)(i, loom)
        for i in tqdm(range(n))
    )

In [23]:
with Loom("parallel_Bline.mp4", fps=10, parallel=True) as loom:
    Parallel(n_jobs=20)(
        delayed(Blineplot_at_i)(i, loom)
        for i in tqdm(range(n))
    )

100%|██████████| 90/90 [03:52<00:00,  2.59s/it]


In [ ]:
with Loom("parallel_primitive.mp4", fps=10, parallel=True) as loom:
    Parallel(n_jobs=20)(
        delayed(plot_at_i)(i, loom)
        for i in tqdm(range(n))
    )

  0%|                                                                                                                                              | 0/90 [00:00<?, ?it/s]

In [ ]:
with Loom("parallel_B.mp4", fps=10, parallel=True) as loom:
    Parallel(n_jobs=20)(
        delayed(Bplot_at_i)(i, loom)
        for i in tqdm(range(n))
    )

100%|██████████| 90/90 [00:25<00:00,  3.60it/s]


KeyboardInterrupt: 